# **Introduction to Artificial Intelligence - MO416A**
**UNIVERSITY OF CAMPINAS**



This work was completed by the following members:



*   Aissa Hadj - 265189
*   Lucas Zanco Ladeira - 188951
*   Matheus Ferraroni - 212142
*   Maria Vitória Rodrigues Oliveira - 262884
*   Oscar Ciceri - 164786

The original code of the project is located on a [repository inside Github](https://github.com/lucaslzl/ga_ia_p2) and the video showing the search strategies working is on [youtube](https://youtube.com). 



# 1 - Introduction



The problem that will be tackled in this project is Feature Selection. The goal is to obtain the subset of available features in a dataset that improves model performance by increasing its accuracy and decreasing its error rate. With the presence of irrelevant features in the dataset, more processing and memory requirements are necessary, thus wasting computing resources. To better understand the possible impacts of feature selection, we could cite the following pros: 

- Reducing Overfitting (nao concordo com esse ponto)
- Improving the model Accuracy
- Reducing Training Time
- removing some features with redundant information
- saving time and costs by collecting fewer features when deploying the model in production
- requiring less computing resources such as memory
- reducing the delay of the model in generating results

Feature Selection may be done manually or automatically. Some manual techniques include: univariate selection, feature importance, and the correlation matrix. The objective of the Univariate selection method is to statistically describe the relation between each feature and the target. Also, feature importance generates a score for each feature in order to rank it. For instance, Decision Tree algorithms may rank features according to Gini impurity tests. Finally, the Correlation Matrix shows the correlation between pairs of features so that the features with a high correlation could be removed. The literature presents the usage of optimization techniques to automatically find the best (or a quite good) subset of features. Some of the methods include:

- <b>Exaustive search</b>
- <b>Simulated Annealing</b>
- <b>Transformation Graph</b>
- <b>Genetic Algorithms</b>

<b>Exaustive Search</b> is not an optimization technique, but it is worth to be mentioned as its computational complexity is $O(2^n)$. This technique tries every possible subset of features in order to find the best one. Due to its computational complexity, this technique is not practical in most cases. <b>Simulated Annealing</b> is a metaheuristic for complex nonlinear optimization problems and is analogous to the simulation of the annealing of solids. The analogy pairs are as follows: feasible solution (state), cost (energy), optimal solution (ground state), local search (rapid quenching), simulated annealing (careful annealing). On the other hand, <b>Transformation Graph</b> is a strategy that utilizes a tree-like structure to generate possible solutions. First, $n$ solutions are generated by removing at each one distinct feature. Second, all the solutions are evaluated. Third, the best solution is chosen and $n-1$ are generated by removing each feature yet not removed. That strategy goes on considering a budget. The issue of this strategy is that it requires a substantial amount of memory. Finally,<b>Genetic Algorithm</b> is inspired by genetics (DNA) to search through solutions. It may be briefly explained with the following steps: it generates a population considering variations of the DNA, then ranks the population according to some score, it applies some forms of mutations and other transformations to the DNA at each generation, and finally, it iterates until it reaches a stopping rule. The stopping rule may be the amount of generations produced or a target solution was reached. This is the strategy applied in this work and will be explained deeply in the next sections.

# 2 - Genetic Algorithm

We developed 2 classes to control the operation of the genetic algorithm. The classes are 'element' and 'GeneticAlgorithm' and they can be seen in https://github.com/lucaslzl/ga_ia_p2/blob/master/GA.py. In this notebook the class 'element' will be examplained, as well as the main methods of the class 'GeneticAlgorithm'.

Below the entire class 'element' is presented. It is possible to see that this class is only responsible for contain the id, the generation, the genome and the score of each element in each generation.

Saving this attributes in the same place can be usefull for different approachs during the implementation of the methods used by the genetic algorithm.

Altogh it's not imeplement here, it is possible to save the parents of each element and traceback how each element was formed during the evolution.

In [0]:
class element:

    def __init__(self, idd, geracao, genome):
        self.idd = idd
        self.geracao = geracao
        self.genome = genome
        self.score = None


    def __repr__(self):
        return "(id="+str(self.idd)+",geracao="+str(self.geracao)+",score="+str(self.score)+")"

The genetic algorithm implemented is very generic, this means that it can be used easily for different problems. Using this kind of generic implementation direct it's use to override the functions responsible for generate a random genome, mutate and fitness calculation.

The method 'create_initial_population' is called once the gentic algorithm is started. This methodo create one element and add it to the pool of this generation till the pool has the size defined. To create the genome the fuction 'random_genome', that was orverride before, is called for each element.

In our problem we define the genome as an array of 0's and 1's with the same length of the amount of t

In [0]:
def create_initial_population(self):
    for _ in range(self.population_size):
        self.population.append(element(self.elements_created, 0, self.random_genome()))
        self.elements_created += 1


def random_genome():
    return np.random.randint(low=0,high=2,size=len(df.columns),dtype=int)

In [0]:
def run(self):

    while self.check_stop():
        self.calculate_score() 
        self.population.sort(key=lambda x: x.score, reverse=True) 

        if self.best_element_total==None or self.population[0].score > self.best_element_total.score: 
            self.best_element_total = self.population[0]

        self.do_log()

        if self.cut_half_population: 
            self.population = self.population[0:len(self.population)//2] 

        self.new_population()

        self.iteration_counter +=1

    return self.best_element_total

In [0]:
def new_population(self):

    probs = self.get_probs()
    newPop = []
    best_replicator = int(self.population_size*self.replicate_best)

    while len(newPop)<self.population_size-best_replicator:
        parents = np.random.choice(self.population,size=2,p=probs) 

        if parents[0].score<parents[1].score: 
            parents = parents[::-1] 

        new_element = element(self.elements_created, self.iteration_counter, self.crossover(parents[0].genome, parents[1].genome))

        new_element.genome = self.active_mutate(new_element.genome)
        newPop.append(new_element)
        self.elements_created += 1

    for i in range(best_replicator):
        newPop.append(self.population[i])

    self.population = newPop

In [0]:
def get_probs(self):
    if self.probs_type == 0:
        return self.probs_roulette()
    elif self.probs_type == 1:
        return self.probs_equal()


def probs_equal(self):
    return [1/len(self.population)]*len(self.population)


def probs_roulette(self):
    probs = [0]*len(self.population) 
    for i in range(len(probs)):
        probs[i] = self.population[i].score
    div = sum(probs)

    if div!=0:
        for i in range(len(probs)):
            probs[i] /= div
    else: 
        probs = self.probs_equal()
    return probs

In [0]:
def check_stop(self):
    if self.stop_criteria_type==0:
        return self.stop_criteria_double()
    elif self.stop_criteria_type==1:
        return self.stop_criteria_iteration()
    elif self.stop_criteria_type==2:
        return self.stop_criteria_score()

def stop_criteria_double(self):
    s = self.population[0].score
    if s==None:
        s = 0
    return self.iteration_counter<self.iteration_limit or s>=self.max_possible_score

def stop_criteria_iteration(self):
    return self.iteration_counter<self.iteration_limit

def stop_criteria_score(self):
    s = self.population[0].score
    if s==None:
        s = 0
    return s>=self.max_possible_score

In [0]:
def crossover(self, genA, genB):
    if self.crossover_type==0:
        return self.crossover_uniform(genA, genB)
    elif self.crossover_type==1:
        return self.crossover_single_point(genA, genB)
    elif self.crossover_type==2:
        return self.crossover_two_point(genA, genB)
    elif self.crossover_type==3:
        return self.crossover_rate_selection(genA, genB)

def crossover_rate_selection(self, genA, genB):
    new = np.array([],dtype=int)
    for i in range(len(genA)):
        if np.random.random()<self.crossover_rate:
            new = np.append(new, genA[i])
        else:
            new = np.append(new, genB[i])
    return new


def crossover_uniform(self, genA, genB):
    new = np.array([],dtype=int)
    for i in range(len(genA)):
        if np.random.random()<0.5:
            new = np.append(new, genA[i])
        else:
            new = np.append(new, genB[i])
    return new


def crossover_single_point(self, genA, genB):
    p = np.random.randint(low=1,high=len(genA)-1) 
    return np.append(genA[0:p],genB[p:])


def crossover_two_point(self, genA, genB):
    c1 = c2 = np.random.randint(low=0,high=len(genA)) 
    while c2==c1: 
        c2 = np.random.randint(low=0,high=len(genA))

    if c1>c2: 
        c1, c2 = c2,c1

    new = np.append(np.append(genA[0:c1],genB[c1:c2]),genA[c2:]) 
    return new

In [0]:
def calculate_score(self):
    if self.use_threads: 

        threads_running = []
        for e in self.population:
            x = threading.Thread(target=self.thread_evaluate, args=(e,))
            x.start()
            threads_running.append(x)

        for i in range(len(threads_running)):
            threads_running[i].join()

    else: 
        for e in self.population:
            e.score = self.evaluate(e.genome)

def thread_evaluate(self, e):
    e.score = self.evaluate(e.genome)
    

def evaluate(genome):
    bool_genome = list(map(bool, genome))
    return model.evaluate(df.loc[:, bool_genome].copy(), target)

In [0]:
def active_mutate(self,gen):
    if self.mutation_rate<=0: 
        return gen
    for i in range(len(gen)): 
        if np.random.random()<self.mutation_rate: 
            gen = self.mutate1(i, gen) 
    return gen


def mutate1(index, genome):
    if genome[index]==0:
        genome[index] = 1
    else:
        genome[index] = 0
    return genome


def mutate2(index, genome):
    aux = []
    for i in range(len(genome)):
        if i <= index:
            aux.append(genome[i])
        else:
            aux.insert(0, genome[i])
    genome = aux
    if genome[index]==0:
        genome[index] = 1
    else:
        genome[index] = 0
    return genome

# 3 - Methodology

# 4 - Results

# 5 - Conclusions